# tf.keras.preprocessing.image_dataset_from_directory
## 1. 基本功能

它会从 目录结构 中自动创建 `tf.data.Dataset`，同时会根据 `子文件夹的名字` 自动生成类别标签。

📂 目录结构示例：

In [ ]:
dataset/
├── train/
│   ├── cats/
│   │   ├── cat1.jpg
│   │   ├── cat2.jpg
│   └── dogs/
│       ├── dog1.jpg
│       ├── dog2.jpg
└── val/
    ├── cats/
    └── dogs/

## 2. 为什么需要使用它？
- 极简代码：用一行代码替代了繁琐的图像读取、解码、调整大小、标注和批处理流程。

- 高性能：生成的是 `tf.data.Dataset` 对象，支持 `TensorFlow` 的所有高效数据管道功能（预取、缓存、并行化等）。

- 内存高效：不会一次性将所有图像加载到内存中，而是动态地从磁盘读取，适合处理大型数据集。

- 自动标签：根据目录结构自动推断标签，无需手动创建标签文件。

---
## 3. 核心参数详解

In [ ]:
tf.keras.preprocessing.image_dataset_from_directory(
    directory,                   # 必需：数据所在目录的路径
    labels='inferred',           # 标签来源：'inferred'（从目录结构），或 None（无标签）
    label_mode='int',            # 标签格式：'int'（整数标签），'categorical'（one-hot），'binary'（二分类）
    class_names=None,            # 显式指定类别名称列表，用于控制顺序
    color_mode='rgb',            # 图像通道：'rgb'（3通道），'grayscale'（1通道），'rgba'（4通道）
    batch_size=32,               # 批次大小
    image_size=(256, 256),       # 将所有图像调整为此尺寸（高度，宽度）
    shuffle=True,                # 是否打乱数据
    seed=None,                   # 随机种子，用于可重现的 shuffling
    validation_split=None,       # 分割部分数据作为验证集（0-1之间的浮点数）
    subset=None,                 # 如果使用 validation_split，指定是 'training' 还是 'validation'
    interpolation='bilinear',    # 图像调整大小的插值方法：'bilinear', 'nearest', 等
    follow_links=False,          # 是否跟随子目录中的符号链接
    crop_to_aspect_ratio=False,  # 是否裁剪图像以保持原始宽高比
    **kwargs
)

### 最重要的参数：
- 1.`directory`：包含按类别分组的子目录的目录路径。

- 2.`image_size`：必须指定。所有图像将被调整为此尺寸。例如 (224, 224)。

- 3.`batch_size`：每个批次中包含的样本数。

- 4.`label_mode`：

    - `int`：标签为整数（如 0, 1, 2）。适用于 `sparse_categorical_crossentropy` 损失。
    
    - `categorical`：标签为 `one-hot` 编码向量（如 [1, 0, 0], [0, 1, 0]）。适用于 `categorical_crossentropy` 损失。
    
    - `binary`：标签为二分类的 0 或 1。适用于 `binary_crossentropy` 损失。

- 5.`validation_split` 和 `subset`：用于从同一目录创建训练/验证集。

---
## 4. 基本使用示例
示例 1：创建训练集和验证集（从不同目录）\
这是最推荐的方式，训练集和验证集分别放在不同的文件夹中。

In [ ]:
import tensorflow as tf

# 定义图像大小和批次大小
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# 从训练目录创建训练集
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory='data/train',      # 训练集目录
    labels='inferred',           # 从目录结构推断标签
    label_mode='categorical',    # 使用 one-hot 编码标签
    color_mode='rgb',            # 彩色图像
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,         # 调整图像大小
    shuffle=True,                # 打乱数据
    seed=123,                    # 随机种子
)

# 从验证目录创建验证集
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory='data/validation', # 验证集目录
    labels='inferred',
    label_mode='categorical',
    color_mode='rgb',
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    shuffle=False,               # 验证集通常不需要打乱
)

# 查看数据集信息
print(f"训练集类别: {train_ds.class_names}")
print(f"训练集批次形状: {next(iter(train_ds))[0].shape}") # (32, 224, 224, 3)
print(f"标签形状: {next(iter(train_ds))[1].shape}")       # (32, 2) - 假设有2个类别

示例 2：从同一目录分割训练/验证集 \
当你的所有数据都在一个目录中时，可以使用 `validation_split` 参数。

In [ ]:
# 创建训练集（80%的数据）
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory='data/all_images',
    validation_split=0.2,        # 使用20%作为验证集
    subset='training',           # 这是训练部分
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
)

# 创建验证集（20%的数据）
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory='data/all_images',
    validation_split=0.2,
    subset='validation',         # 这是验证部分
    seed=123,                    # 必须使用相同的seed！
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
)

## 5 遍历数据集
使用 数据集 `.take(n)` 从数据集中取出前 `n` 个批次的数据（n是你传入的数字）

In [ ]:
for images, labels in train_ds.take(1):
    print(images.shape)  # (32, 224, 224, 3)
    print(labels.numpy())  # e.g. [0, 1, 0, ...]


## 6. 数据预处理和增强
创建数据集后，你通常需要添加预处理和数据增强。

标准预处理（归一化）

In [ ]:
# 定义预处理函数：将像素值从 [0, 255] 归一化到 [0, 1] 或 [-1, 1]
def preprocess(image, label):
    # 方法1: 归一化到 [0, 1]
    image = tf.cast(image, tf.float32) / 255.0
    
    # 方法2: 归一化到 [-1,  ]（某些预训练模型需要）
    # image = (tf.cast(image, tf.float32) / 127.5) - 1
    
    return image, label

# 应用预处理
train_ds = train_ds.map(preprocess)
val_ds = val_ds.map(preprocess)

In [ ]:
# 创建数据增强层
data_augmentation = tf.keras.Sequential([
    # 会随机对输入图像进行水平翻转、垂直翻转，或者同时进行水平和垂直翻转（即对角线翻转）
    tf.keras.layers.RandomFlip("horizontal_and_vertical"),
    tf.keras.layers.RandomRotation(0.2), # 用于对输入图像进行随机旋转
    tf.keras.layers.RandomZoom(0.1),     # 用于对输入图像进行随机缩放（放大或缩小）
    tf.keras.layers.RandomContrast(0.1), # 用于对输入图像进行随机对比度调整
])

# 应用数据增强（只对训练集）
def augment(image, label):
    image = data_augmentation(image, training=True)
    return image, label

train_ds = train_ds.map(augment)

## 6. 性能优化
使用 `tf.data` 的性能优化技巧来加速训练：

In [ ]:
# 配置数据集以获得最佳性能
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000) # 将数据缓存到内存中（如果数据集较小）.shuffle(1000) → 打乱
train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)  # 在训练时预取后续批次

val_ds = val_ds.cache()
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)

## 7. 完整的工作流程示例 

In [ ]:
import tensorflow as tf

# 1. 创建数据集
IMG_SIZE = (180, 180)
BATCH_SIZE = 32

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    'data/train',
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='categorical'
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    'data/train',
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='categorical'
)

# 查看类别信息
class_names = train_ds.class_names
print(f"类别: {class_names}")
print(f"类别数量: {len(class_names)}")

# 2. 数据预处理和增强
# 归一化
def preprocess(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

# 数据增强
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
])

def augment(image, label):
    image = data_augmentation(image, training=True)
    return image, label

# 应用预处理和增强
train_ds = train_ds.map(augment).map(preprocess)
val_ds = val_ds.map(preprocess)

# 3. 性能优化
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

# 4. 创建和训练模型
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(len(class_names), activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# 5. 训练模型
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)

## 8.与 ImageDataGenerator 的区别
| `image_dataset_from_directory`  | `ImageDataGenerator` |
| ------------------------------- | -------------------- |
| 返回 `tf.data.Dataset`（适合 TF 2.x） | 返回 Python 生成器        |
| 更快、更优化                          | 功能较旧                 |
| 支持 GPU pipeline                 | 更多传统数据增强             |
